**CS6700: Reinforcement Learning**

*Assignment 3*

Team Members
- Prajwal Shettigar J (CH22D302)
- Akash Sharma (EE21S056)

# Description of Options

The following description is for the default set of options that are used in this assignment. The implementation closely follows the formulation in [Between MDPs and semi-MDPs : A framework for temporal abstraction in reinforcement learning](https://people.cs.umass.edu/~barto/courses/cs687/Sutton-Precup-Singh-AIJ99.pdf). 

1. **Primitive Options** : Actions can be considered as a special case of options which only execute for a single step, terminate after the taking the said step and have the state space itself as the initiation set. 

2. **Multi-step Options** : Deterministic policies for the set of temporally extended options (`Opt_R`, `Opt_G`, `Opt_Y` and `Opt_B`) are used as is the case in the illustration example in the reference mentioned above. These policies are defined in the code cell below. Each location in the `Taxi-v3` environment grid is assigned one of the four movement actions i.e., `South` , `North`, `East` and `West` corresponding to integers $0$, $1$, $2$ and $3$. These options terminate at their respective locations as described in the environment and the options execute only if they are not present at these locations.

These options were chosen by *policy over options*, for this case epsilon-greedy policy was used.

In [ ]:
'''
Primitive options are special cases of options which last exactly one step.
Primitive options are South, North, East, West, Pick, Drop.
Rest of the options defined are multi-step i.e., temporally extended.
Temporally extended options are Opt_R, Opt_G, Opt_Y, Opt_B

These options correspond to [0,1,2,3,4,5,6,7,8,9] respectively.
'''

default_options = ["South", "North", "East", "West", "Pick", "Drop", 
           "Opt_R", "Opt_G", "Opt_Y", "Opt_B"]

# 0: South, 1: North, 2: East, 3: West
# Here, the policy is defined as (x,y) location-action pair, this will be translated to state-action pair for options
# Deterministic policy which enables the agent to get goal R

Opt_R_policy = np.array([[1,3,0,0,0],
                         [1,3,0,0,0],
                         [1,3,3,3,3],
                         [1,1,1,1,1],
                         [1,1,1,1,1]])

# Deterministic policy which enables the agent to get goal G
Opt_G_policy = np.array([[0,0,2,2,1],
                         [0,0,2,2,1],
                         [2,2,2,1,1],
                         [1,2,1,1,1],
                         [1,2,1,1,1]])

# Deterministic policy which enables the agent to get goal Y
Opt_Y_policy = np.array([[0,3,0,0,0],
                         [0,3,0,0,0],
                         [0,3,3,3,3],
                         [0,1,1,1,3],
                         [0,1,3,1,3]])

# Deterministic policy which enables the agent to get goal B
Opt_B_policy = np.array([[0,0,0,0,3],
                         [0,0,0,0,3],
                         [2,2,2,0,3],
                         [1,1,1,0,3],
                         [1,1,1,0,3]])

# SMDP Q-Learning

In SMDP Q-Learning, all the options were used as in `default_options` shown in the code snippet above. The Q-value update equation for temporally extended option in SMDP Q-Learning needs to calculated where a series of actions is taken according to the policy of that particular option. If an option is chosen by policy over options is non-executable, then there is no change in the state and no reward is incurred. A code snippet from the function `smdp_ql_taxi` is shown below where the Q-value update for a multistep option is shown. The `multistep_opt` function is also shown which takes care of the calculation of the cumulative reward and moves the taxi according the option's deterministic policy.

In [ ]:
### CODE SNIPPET FROM smdp_ql_taxi for Q-value update of multi-step option ###

# Check if state is part of initiation set and decide if the option has to be executed or not
execute_option = opt_execute(option, state, environment)
if execute_option:
    next_state, cumulative_reward, timesteps_elapsed, done = multistep_opt_config(option, state, environment, gamma) # r(s,o) is cumulative reward for multi-step options
    # One-step SMDP Q-Learning update since multi-step option
    # Q(s,o) update for multi-step option
    Q_values[state, option] += alpha*(cumulative_reward + (gamma**timesteps_elapsed)*(max(Q_values[next_state,:])-
                                                                                        Q_values[state, option]))
    state = next_state
    episode_reward += cumulative_reward
else: 
    state = next_state
    done = False # done remains False since the state has not changed


### CODE for multistep_opt function ###

def multistep_opt_default(option, state, environment, gamma):
    opt_done = False
    goal_locations = {6:(0,0), 7:(0,4), 8:(4,0), 9:(4,3)} # 6,7,8,9 correspond to R,G,Y,B respectively
    option_policy_mapping = {6: Opt_R_policy, 7: Opt_G_policy, 8: Opt_Y_policy, 9: Opt_B_policy}
    cumulative_reward = 0
    timestep_count = 0
    while not opt_done:
        taxi_row, taxi_col, _, _ = list(environment.decode(state))
        action = option_policy_mapping[option][taxi_row][taxi_col]
        next_state, reward, done, _, _ = environment.step(action)
        timestep_count += 1
        cumulative_reward += (gamma**(timestep_count-1))*reward
        state = next_state
        taxi_row, taxi_col, _, _ = list(environment.decode(state))
        if (taxi_row, taxi_col) == goal_locations[option]:
            opt_done = True

## Results

The algorithm was run with the configuration given below. The reward curve and Q-table is also shown. 

## Description of the policy learnt

# Intra-Option Q-Learning

## Code Snippet

## Results

## Description of the policy learnt

# Comparison with mutually exclusive set of options

## Code snippet

Compare the performance with other set of options

# Comparison between SMDP Q-Learning & Intra-Option Q-Learning